In [ ]:
import os
import json
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
project_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(project_directory, "data")
dataset_directory = os.path.join(project_directory, "tf_data")
model_directory = os.path.join(project_directory, "models")
model_name = "model_v2" # Change this

img_height = 200
img_width = 200
batch_size = 32

class_names = sorted([name for name in os.listdir(dataset_directory) if os.path.isdir(os.path.join(dataset_directory, name))])

Load data

In [ ]:
model = tf.keras.models.load_model(os.path.join(model_directory, f'{model_name}.keras'))
with open(os.path.join(model_directory, f'{model_name}_history.json'), 'r') as f:
    history = json.load(f)

Create a summary of the layers and training parameters

In [ ]:
model.summary()

Plot training & validation loss values

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

Create classification report

In [ ]:
y_true = []
y_pred = []
random_seat = 123

val_ds = image_dataset_from_directory(
    dataset_directory,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

def extract_channel(image, channel=0):
    # channel can be 0 (red), 1 (green), or 2 (blue)
    return image[:, :, :, channel:channel+1]

val_ds = val_ds.map(lambda x, y: (extract_channel(x), y))

for images, labels in val_ds:
    predictions = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(predictions, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

conf_matrix = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

Plot first batch with predictions

In [ ]:
val_ds_shuffled = val_ds.shuffle(buffer_size=len(val_ds))

for images, labels in val_ds_shuffled.take(1):
    predictions = model.predict(images)
    
    plt.figure(figsize=(10, 10))
    for i in range(len(images)):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"), cmap="gray")
        predicted_class = np.argmax(predictions[i])
        confidence = np.max(predictions[i])
        true_label = class_names[labels[i]]
        predicted_label = class_names[predicted_class]
        color = "green" if predicted_label == true_label else "red"
        plt.title(f"True: {true_label}\nPred: {predicted_label} ({confidence:.2f})", color=color)
        plt.axis("off")
    plt.show()